# Testando as imagens iniciais

## Usando Yolov3 e COCO dataset

In [ ]:
# Use wget to download the YOLOv3 configuration file
!wget https://github.com/pjreddie/darknet/blob/master/cfg/yolov3.cfg?raw=true -O yolov3.cfg

# Use wget to download the YOLOv3 weights file
!wget https://pjreddie.com/media/files/yolov3.weights

# Use wget to download the coco.names file
!wget https://github.com/pjreddie/darknet/blob/master/data/coco.names?raw=true -O coco.names

In [ ]:
from google.colab.patches import cv2_imshow
import cv2
import numpy as np
import os
current_directory = os.getcwd()
print(current_directory)

In [ ]:
def detect_objects(image_path):
    # Load YOLO
    net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")  # Load pre-trained weights and configuration
    classes = []
    with open("coco.names", "r") as f:
        classes = [line.strip() for line in f.readlines()]

    # Get the output layer names directly from the network
    layer_names = net.getUnconnectedOutLayersNames()

    # Load image
    img = cv2.imread(image_path)
    img = cv2.resize(img, None, fx=0.4, fy=0.4)  # Resize image for faster processing
    height, width, channels = img.shape

    # Detecting objects
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(layer_names)

    # Information to display on the image
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    # Draw bounding boxes and labels on the image
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = confidences[i]
            color = (0, 255, 0)  # Green
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
            cv2.putText(img, f"{label} {confidence:.2f}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Display the image with detected objects
    cv2_imshow(img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
def apply_detector(path):
  for filename in os.listdir(path):
    if filename.lower().endswith(".png"):
        #image_path = os.path.join(directory_path, filename)
        print(filename)
        detect_objects(filename)


In [ ]:
apply_detector('/content/')